<a href="https://colab.research.google.com/github/wayoo2019/Time-Series-analysis/blob/main/475_Project3_1_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Q2.1

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import numpy as np
import pandas as pd
import unicodedata
import string
import torch
import torch.nn as nn
import random
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [ ]:
def findFiles(path): 
    return glob.glob(path)

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )


names = {}
languages = []


def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

# (TO DO:) CHANGE FILE PATH AS NECESSARY
for filename in findFiles('/content/drive/MyDrive/475 Time Series/Project 3/data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    languages.append(category)
    lines = readLines(filename)
    names[category] = lines


n_categories = len(languages)

def letterToIndex(letter):
    return all_letters.find(letter)


def nameToTensor(name):
    tensor = torch.zeros(len(name), 1, n_letters)
    for li, letter in enumerate(name):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor


class RNN(nn.Module):
    def __init__(self, INPUT_SIZE, HIDDEN_SIZE, N_LAYERS,OUTPUT_SIZE):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(
            input_size = INPUT_SIZE,
            hidden_size = HIDDEN_SIZE, # number of hidden units
            num_layers = N_LAYERS, # number of layers
            batch_first = True)
        self.out = nn.Linear(HIDDEN_SIZE, OUTPUT_SIZE)
            
    def forward(self, x):
        r_out, h = self.rnn(x, None) # None represents zero initial hidden state           
        out = self.out(r_out[:, -1, :])
        return out

n_hidden = 128

allnames = [] # Create list of all names and corresponding output language
for language in list(names.keys()):
    for name in names[language]:
        allnames.append([name, language])
        
## (TO DO:) Determine Padding length (this is the length of the longest string) 

# maxlen = ..... # Add code here to compute the maximum length of string 
maxlen = 0
for name in allnames:
  if maxlen <= len(nameToTensor(name[0])):
    maxlen = len(nameToTensor(name[0]))
padded_length = maxlen

                
n_letters = len(all_letters)
n_categories = len(languages)

def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i.item()
    return languages[category_i], category_i

learning_rate = 0.005
rnn = RNN(n_letters, 128, 1, n_categories)
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)   # optimize all rnn parameters
loss_func = nn.CrossEntropyLoss()  

for epoch in range(5):  
    batch_size = len(allnames)
    random.shuffle(allnames)
    
    # if "b_in" and "b_out" are the variable names for input and output tensors, you need to create those
    
# (TO DO:) Initialize "b_in" to a tensor with size of input (batch size, padded_length, n_letters)
# (TO DO:) Initialize "b_out" to tensor with size (batch_size, n_categories, dtype=torch.long)   
    b_in = torch.zeros(batch_size, padded_length, n_letters)
    b_out = torch.zeros(batch_size, n_categories, dtype=torch.long)

    # (TO DO:) Populate "b_in" tensor 
    # (TO DO:) Populate "b_out" tensor
    for i, name in enumerate(allnames):
      for li, letter in enumerate(name[0]):
        b_in[i][li][letterToIndex(letter)] = 1
    
    for i, name in enumerate(allnames): 
      b_out[i][languages.index(name[1])] = 1
    
       
    output = rnn(b_in)                               # rnn output
    #(TO DO:)
    loss = loss_func(output, torch.max(b_out, 1)[1])   # (TO DO:) Fill "...." to calculate the cross entropy loss
    optimizer.zero_grad()                           # clear gradients for this training step
    loss.backward()                                 # backpropagation, compute gradients
    optimizer.step()                                # apply gradients
        
    # Print accuracy
    test_output = rnn(b_in)                   # 
    pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
    test_y = torch.max(b_out, 1)[1].data.numpy().squeeze()
    accuracy = sum(pred_y == test_y)/batch_size
    print("Epoch: ", epoch, "| train loss: %.4f" % loss.item(), '| accuracy: %.2f' % accuracy)
    


Epoch:  0 | train loss: 2.8475 | accuracy: 0.47
Epoch:  1 | train loss: 2.6160 | accuracy: 0.47
Epoch:  2 | train loss: 2.0862 | accuracy: 0.47
Epoch:  3 | train loss: 1.9306 | accuracy: 0.47
Epoch:  4 | train loss: 1.9574 | accuracy: 0.47


#Q2.2

**Batch size = 1000:**

In [ ]:
BATCH_SIZE = 1000 # assign batch size in the beginning
maxlen = 0
for name in allnames[:BATCH_SIZE]:
  if maxlen <= len(nameToTensor(name[0])):
    maxlen = len(nameToTensor(name[0]))
padded_length = maxlen

for epoch in range(5):  
    batch_size = BATCH_SIZE
    random.shuffle(allnames)

    b_in = torch.zeros(batch_size, padded_length, n_letters)
    b_out = torch.zeros(batch_size, n_categories, dtype=torch.long)

    for i, name in enumerate(allnames):
      for li, letter in enumerate(name[0]):
        if i < BATCH_SIZE:
          b_in[i][li][letterToIndex(letter)] = 1
    
    for i, name in enumerate(allnames): 
      if i < BATCH_SIZE:
        b_out[i][languages.index(name[1])] = 1
    
       
    output = rnn(b_in)                               # rnn output
    #(TO DO:)
    loss = loss_func(output, torch.max(b_out, 1)[1])   # (TO DO:) Fill "...." to calculate the cross entropy loss
    optimizer.zero_grad()                           # clear gradients for this training step
    loss.backward()                                 # backpropagation, compute gradients
    optimizer.step()                                # apply gradients
        
    # Print accuracy
    test_output = rnn(b_in)                   # 
    pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
    test_y = torch.max(b_out, 1)[1].data.numpy().squeeze()
    accuracy = sum(pred_y == test_y)/batch_size
    print("Epoch: ", epoch, "| train loss: %.4f" % loss.item(), '| accuracy: %.2f' % accuracy)


Epoch:  0 | train loss: 1.7807 | accuracy: 0.49
Epoch:  1 | train loss: 1.8961 | accuracy: 0.46
Epoch:  2 | train loss: 1.8071 | accuracy: 0.48
Epoch:  3 | train loss: 1.8563 | accuracy: 0.47
Epoch:  4 | train loss: 1.8922 | accuracy: 0.44


**Bacth size = 2000:**

In [ ]:
BATCH_SIZE = 2000 # assign batch size in the beginning
maxlen = 0
for name in allnames[:BATCH_SIZE]:
  if maxlen <= len(nameToTensor(name[0])):
    maxlen = len(nameToTensor(name[0]))
padded_length = maxlen

for epoch in range(5):  
    batch_size = BATCH_SIZE
    random.shuffle(allnames)

    b_in = torch.zeros(batch_size, padded_length, n_letters)
    b_out = torch.zeros(batch_size, n_categories, dtype=torch.long)

    for i, name in enumerate(allnames):
      for li, letter in enumerate(name[0]):
        if i < BATCH_SIZE:
          b_in[i][li][letterToIndex(letter)] = 1
    
    for i, name in enumerate(allnames): 
      if i < BATCH_SIZE:
        b_out[i][languages.index(name[1])] = 1
    
       
    output = rnn(b_in)                               # rnn output
    #(TO DO:)
    loss = loss_func(output, torch.max(b_out, 1)[1])   # (TO DO:) Fill "...." to calculate the cross entropy loss
    optimizer.zero_grad()                           # clear gradients for this training step
    loss.backward()                                 # backpropagation, compute gradients
    optimizer.step()                                # apply gradients
        
    # Print accuracy
    test_output = rnn(b_in)                   # 
    pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
    test_y = torch.max(b_out, 1)[1].data.numpy().squeeze()
    accuracy = sum(pred_y == test_y)/batch_size
    print("Epoch: ", epoch, "| train loss: %.4f" % loss.item(), '| accuracy: %.2f' % accuracy)

Epoch:  0 | train loss: 1.8425 | accuracy: 0.46
Epoch:  1 | train loss: 1.8707 | accuracy: 0.45
Epoch:  2 | train loss: 1.8620 | accuracy: 0.47
Epoch:  3 | train loss: 1.8476 | accuracy: 0.46
Epoch:  4 | train loss: 1.8302 | accuracy: 0.47


**Bacth size = 3000:**

In [ ]:
BATCH_SIZE = 3000 # assign batch size in the beginning
maxlen = 0
for name in allnames[:BATCH_SIZE]:
  if maxlen <= len(nameToTensor(name[0])):
    maxlen = len(nameToTensor(name[0]))
padded_length = maxlen

for epoch in range(5):  
    batch_size = BATCH_SIZE
    random.shuffle(allnames)

    b_in = torch.zeros(batch_size, padded_length, n_letters)
    b_out = torch.zeros(batch_size, n_categories, dtype=torch.long)

    for i, name in enumerate(allnames):
      for li, letter in enumerate(name[0]):
        if i < BATCH_SIZE:
          b_in[i][li][letterToIndex(letter)] = 1
    
    for i, name in enumerate(allnames): 
      if i < BATCH_SIZE:
        b_out[i][languages.index(name[1])] = 1
    
       
    output = rnn(b_in)                               # rnn output
    #(TO DO:)
    loss = loss_func(output, torch.max(b_out, 1)[1])   # (TO DO:) Fill "...." to calculate the cross entropy loss
    optimizer.zero_grad()                           # clear gradients for this training step
    loss.backward()                                 # backpropagation, compute gradients
    optimizer.step()                                # apply gradients
        
    # Print accuracy
    test_output = rnn(b_in)                   # 
    pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
    test_y = torch.max(b_out, 1)[1].data.numpy().squeeze()
    accuracy = sum(pred_y == test_y)/batch_size
    print("Epoch: ", epoch, "| train loss: %.4f" % loss.item(), '| accuracy: %.2f' % accuracy)

Epoch:  0 | train loss: 1.8782 | accuracy: 0.46
Epoch:  1 | train loss: 1.8222 | accuracy: 0.48
Epoch:  2 | train loss: 1.8957 | accuracy: 0.46
Epoch:  3 | train loss: 1.8637 | accuracy: 0.47
Epoch:  4 | train loss: 1.8502 | accuracy: 0.47
